In [1]:
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# import tensorflow as tf


#定義常量
rnn_unit=10       #hidden layer units
input_size=7
output_size=1
lr=0.0006         #學習率
#——————————————————導入數據——————————————————————
f=open('/content/drive/MyDrive/Colab Notebooks/stock_predict/data_set(0616).csv') 
df=pd.read_csv(f)     #讀入股票資料
data=df.iloc[:2560,2:20].values  #取第3-19列
print('data shape:',data.shape)




data shape: (2560, 18)


In [64]:
#獲取訓練集
def get_train_data(batch_size=60,time_step=20,train_begin=0,train_end=2000):
    batch_index=[]
    data_train=data[train_begin:train_end]
    normalized_train_data=(data_train-np.mean(data_train,axis=0))/np.std(data_train,axis=0)  #標準化
    train_x,train_y=[],[]   #訓練集 
 
    for i in range(len(normalized_train_data)-time_step):
      if i % batch_size==0:
        batch_index.append(i)
      x=normalized_train_data[i:i+time_step,:17]
      y=normalized_train_data[i:i+time_step,17,np.newaxis]
      train_x.append(x.tolist())
      train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    return batch_index,train_x,train_y


In [65]:
#獲取測試集
def get_test_data(time_step=20,test_begin=2000):
    data_test=data[test_begin:]
    mean=np.mean(data_test,axis=0)
    std=np.std(data_test,axis=0)
    normalized_test_data=(data_test-mean)/std  #標準化
    # normalized_test_data=(data_test-np.mean(data_test,axis=0))/std  #標準化
    print(len(normalized_test_data))
    size=(len(normalized_test_data)+time_step-1)//time_step  #有size個sample
    # print(len(normalized_test_data)//time_step)
    # print(size) 
    test_x,test_y=[],[]
    for i in range(size-1):
      # print((i+1)*time_step)
      x=normalized_test_data[i*time_step:(i+1)*time_step,:17]
      y=normalized_test_data[i*time_step:(i+1)*time_step,10]
      test_x.append(x.tolist())
      test_y.extend(y)
    test_x.append((normalized_test_data[(i+1)*time_step:,:17]).tolist())
    test_y.extend((normalized_test_data[(i+1)*time_step:,10]).tolist())
    return mean,std,test_x,test_y


In [66]:


#——————————————————定義神經網路變數——————————————————
#輸入層、輸出層權重、偏置

weights={
         'in':tf.Variable(tf.random.uniform([input_size,rnn_unit])),
         'out':tf.Variable(tf.random.uniform([rnn_unit,1]))
        }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[rnn_unit,])),
        'out':tf.Variable(tf.constant(0.1,shape=[1,]))
       }

#——————————————————定義神經網路變數——————————————————
def lstm(X):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']  
    input=tf.reshape(X,[-1,input_size])  #需要將tensor轉成2維進行計算，計算後的結果作為隱藏層的輸入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #將tensor轉成3維，作為lstm cell的輸入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)

    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是記錄lstm每個輸出節點的結果，final_states是最後一個cell的結果
    tf.reset_default_graph()   ##debug

    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作為輸出層的輸入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states


#——————————————————訓練模型——————————————————
def train_lstm(batch_size=80,time_step=15,train_begin=2000,train_end=5800):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step,train_begin,train_end)
    pred,_=lstm(X)
    #損失函數
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
    module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        #sess.run(tf.global_variables_initializer())
        saver.restore(sess, module_file)
        #重複訓練10000次
        for i in range(2000):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]]})
            print(i,loss_)
            if i % 200==0:
                print("保存模型：",saver.save(sess,'stock2.model',global_step=i))

train_lstm()

#————————————————預測模型————————————————————
def prediction(time_step=20):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    #Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    mean,std,test_x,test_y=get_test_data(time_step)
    pred,_=lstm(X)     
    saver=tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        #參數恢復
        module_file = tf.train.latest_checkpoint()
        saver.restore(sess, module_file) 
        test_predict=[]
        for step in range(len(test_x)-1):
          prob=sess.run(pred,feed_dict={X:[test_x[step]]})   
          predict=prob.reshape((-1))
          test_predict.extend(predict)
        test_y=np.array(test_y)*std[7]+mean[7]
        test_predict=np.array(test_predict)*std[7]+mean[7]
        acc=np.average(np.abs(test_predict-test_y[:len(test_predict)])/test_y[:len(test_predict)])  #偏差
        #以折線圖表示結果
        plt.figure()
        plt.plot(list(range(len(test_predict))), test_predict, color='b')
        plt.plot(list(range(len(test_y))), test_y,  color='r')
        plt.show()

prediction() 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:708: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


ValueError: ignored